In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../data/breadt_football_game_list.csv', parse_dates=['time'])
refer = pd.read_csv('../data/breadt_football_refer_games.csv', parse_dates=['date'])

refer.head()

,id,fid,pos,gd,gs,gn,result,home_team,visit_team,name,date
0,1,331954,hr10g,6,3,9,0,马来明星,利物浦,球会友谊,2011-07-16
1,2,331954,hr10g,2,0,2,0,马来明星,曼联,球会友谊,2009-07-20
2,3,331954,hr10g,4,0,4,0,马来明星,阿森纳,球会友谊,2011-07-13
3,4,331954,hr10g,3,2,5,0,马来明星,曼联,球会友谊,2009-07-18
4,5,331954,vr10g,1,0,1,0,朴茨茅,切尔西,球会友谊,2011-07-16


In [5]:
refer.groupby('pos').size()

pos
ex       375784
hr10g    787006
hrhg     785586
vr10g    787058
vrvg     785682
dtype: int64

In [6]:
def fetch_data(df, refer, pos, result, col_name):
    refer['fid'] = refer['fid'].astype(np.int64)
    ex_lose = refer[(refer['pos'] == pos) & (refer['result'] == result)]
    ex_lose = ex_lose.groupby(['fid'])['result'].size().to_frame()
    ex_lose_group = pd.DataFrame({'fid': ex_lose.index.values, col_name: ex_lose['result'].values})

    df = pd.merge(df, ex_lose_group, on=['fid'], how='left')
    df[col_name] = df[col_name].fillna(0)
    
    return df

In [59]:
def fetch_goal_data(df, refer, pos, prefix):
    refer['fid'] = refer['fid'].astype(np.int64)
    ex_lose = refer[(refer['pos'] == pos)]
    
    gd = ex_lose.groupby(['fid'])['gd'].sum().to_frame()
    gs = ex_lose.groupby(['fid'])['gs'].sum().to_frame()
    
    ex_lose_group = pd.DataFrame({'fid': gd.index.values, prefix + '_gd': gd['gd'].values, prefix + '_gs': gs['gs'].values})

    df = pd.merge(df, ex_lose_group, on=['fid'], how='left')
    df[prefix + '_gd'] = df[prefix + '_gd'].fillna(0)
    df[prefix + '_gs'] = df[prefix + '_gs'].fillna(0)
    
    return df

In [7]:
df = fetch_data(df, refer, 'ex', 0, 'ex_lose_num')
df = fetch_data(df, refer, 'ex', 1, 'ex_draw_num')
df = fetch_data(df, refer, 'ex', 2, 'ex_win_num')

In [8]:
df = fetch_data(df, refer, 'hr10g', 0, 'hr10g_lose_num')
df = fetch_data(df, refer, 'hr10g', 1, 'hr10g_draw_num')
df = fetch_data(df, refer, 'hr10g', 2, 'hr10g_win_num')

In [9]:
df = fetch_data(df, refer, 'hrhg', 0, 'hrhg_lose_num')
df = fetch_data(df, refer, 'hrhg', 1, 'hrhg_draw_num')
df = fetch_data(df, refer, 'hrhg', 2, 'hrhg_win_num')

In [10]:
df = fetch_data(df, refer, 'vr10g', 0, 'vr10g_lose_num')
df = fetch_data(df, refer, 'vr10g', 1, 'vr10g_draw_num')
df = fetch_data(df, refer, 'vr10g', 2, 'vr10g_win_num')

In [49]:
df = fetch_data(df, refer, 'vrvg', 0, 'vrvg_lose_num')
df = fetch_data(df, refer, 'vrvg', 1, 'vrvg_draw_num')
df = fetch_data(df, refer, 'vrvg', 2, 'vrvg_win_num')

In [60]:
df = fetch_goal_data(df, refer, 'ex', 'ex')
df = fetch_goal_data(df, refer, 'hr10g', 'hr10g')
df = fetch_goal_data(df, refer, 'hrhg', 'hrhg')
df = fetch_goal_data(df, refer, 'vr10g', 'vr10g')
df = fetch_goal_data(df, refer, 'vrvg', 'vrvg')

In [64]:
df['ex_diff'] = df['ex_gs'] - df['ex_gd'] 
df['hr10g_diff'] = df['hr10g_gs'] - df['hr10g_gd'] 
df['hrhg_diff'] = df['hrhg_gs'] - df['hrhg_gd'] 
df['vr10g_diff'] = df['vr10g_gs'] - df['vr10g_gd'] 
df['vrvg_diff'] = df['vrvg_gs'] - df['vrvg_gd'] 

In [65]:
df.columns

Index(['id', 'fid', 'status', 'game', 'turn', 'home_team', 'visit_team', 'gs',
       'gd', 'gn', 'offset', 'time', 'result', 'ex_lose_num', 'ex_draw_num',
       'ex_win_num', 'hr10g_lose_num', 'hr10g_draw_num', 'hr10g_win_num',
       'hrhg_lose_num', 'hrhg_draw_num', 'hrhg_win_num', 'vr10g_lose_num',
       'vr10g_draw_num', 'vr10g_win_num', 'year', 'month', 'day',
       'home_team_encoder', 'visit_team_encoder', 'game_encoder',
       'vrvg_lose_num', 'vrvg_draw_num', 'vrvg_win_num', 'ex_gd', 'ex_gs',
       'hr10g_gd', 'hr10g_gs', 'hrhg_gd', 'hrhg_gs', 'vr10g_gd', 'vr10g_gs',
       'vrvg_gd', 'vrvg_gs', 'ex_diff', 'hr10g_diff', 'hrhg_diff',
       'vr10g_diff', 'vrvg_diff'],
      dtype='object')

In [11]:
df['year'] = df.apply(lambda row: row.time.year, axis=1)
df['month'] = df.apply(lambda row: row.time.month, axis=1)
df['day'] = df.apply(lambda row: row.time.day, axis=1)

In [12]:
df = df[df['status'] == '完']

In [13]:
from sklearn import preprocessing

team_arr = list(set(df['home_team'].values) | set(df['visit_team'].values))

team_encoder = preprocessing.LabelEncoder()
team_encoder.fit(team_arr)

df['home_team_encoder'] = team_encoder.transform(df['home_team'])
df['visit_team_encoder'] = team_encoder.transform(df['visit_team'])

In [14]:
game_arr = list(set(df['game'].values))

game_encoder = preprocessing.LabelEncoder()
game_encoder.fit(game_arr)

df['game_encoder'] = game_encoder.transform(df['game'])

# xgb

In [68]:
import xgboost as xgb

# x_columns = ['fid', 'year', 'month', 'day', 'game_encoder', 'visit_team_encoder', 'home_team_encoder', 'ex_lose_num', 'ex_draw_num', 'ex_win_num', 'hr10g_lose_num', 'hr10g_draw_num', 'hr10g_win_num', 'hrhg_lose_num', 'hrhg_draw_num', 'hrhg_win_num','vr10g_lose_num', 'vr10g_draw_num','vr10g_win_num', 'vrvg_lose_num', 'vrvg_draw_num', 'vrvg_win_num']
x_columns = ['game_encoder', 'visit_team_encoder', 'home_team_encoder', 'ex_lose_num', 'ex_draw_num', 'ex_win_num', 'hr10g_lose_num', 'hr10g_draw_num', 'hr10g_win_num', 'hrhg_lose_num', 'hrhg_draw_num', 'hrhg_win_num','vr10g_lose_num', 'vr10g_draw_num','vr10g_win_num', 'vrvg_lose_num', 'vrvg_draw_num', 'vrvg_win_num', 'ex_gd', 'ex_gs',
       'hr10g_gd', 'hr10g_gs', 'hrhg_gd', 'hrhg_gs', 'vr10g_gd', 'vr10g_gs',
       'vrvg_gd', 'vrvg_gs', ]
             
#              'ex_diff', 'hr10g_diff', 'hrhg_diff',
#        'vr10g_diff', 'vrvg_diff']


params={
'booster':'gbtree',
# 这里手写数字是0-9，是一个多类的问题，因此采用了multisoft多分类器，
'objective': 'multi:softmax', 
'num_class':3, # 类数，与 multisoftmax 并用
'gamma':0.1,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 。[0:]
'max_depth':12, # 构建树的深度 [1:]
#'lambda':450,  # L2 正则项权重
'subsample':0.7, # 采样训练数据，设置为0.5，随机选择一般的数据实例 (0:1]
'colsample_bytree':0.7, # 构建树树时的采样比率 (0:1]
#'min_child_weight':12, # 节点的最少特征数
'silent':1 ,
'eta': 0.05, # 如同学习率
'seed':710,
'nthread':4,# cpu 线程数,根据自己U的个数适当调整
}

train_dataset = df[df['year'] < 2019]
test_dataset = df[df['year'] == 2019]

xgtrain = xgb.DMatrix(train_dataset[x_columns], label=train_dataset['result'])
xgtest = xgb.DMatrix(test_dataset[x_columns], label=test_dataset['result'])

watchlist = [(xgtrain, 'train'),(xgtest, 'val')]

# num_rounds = 10000
# stop_rounds = 100

num_rounds = 500
stop_rounds = 50

model = xgb.train(params, xgtrain, num_rounds, watchlist,early_stopping_rounds=stop_rounds)
print(model.best_iteration)

preds = model.predict(xgtest, ntree_limit=model.best_iteration)
preds

[0]	train-merror:0.449308	val-merror:0.56632
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 50 rounds.
[1]	train-merror:0.417983	val-merror:0.550715
[2]	train-merror:0.405549	val-merror:0.550715
[3]	train-merror:0.400175	val-merror:0.547464
[4]	train-merror:0.394789	val-merror:0.544863
[5]	train-merror:0.391001	val-merror:0.541613
[6]	train-merror:0.390266	val-merror:0.542913
[7]	train-merror:0.389107	val-merror:0.542263
[8]	train-merror:0.38765	val-merror:0.536411
[9]	train-merror:0.386774	val-merror:0.540962
[10]	train-merror:0.385769	val-merror:0.539012
[11]	train-merror:0.384932	val-merror:0.539662
[12]	train-merror:0.381362	val-merror:0.537061
[13]	train-merror:0.37903	val-merror:0.536411
[14]	train-merror:0.376221	val-merror:0.53186
[15]	train-merror:0.374417	val-merror:0.528609
[16]	train-merror:0.372239	val-merror:0.538361
[17]	train-merror:0.370667	val-merror:0.53186
[18]	train-merror:0.3682

array([2., 0., 2., ..., 2., 0., 2.], dtype=float32)

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(test_dataset['result'], preds)

0.4687906371911573